
# Multi‑Plan 401(k) RAG with AWS Bedrock + FAISS (Per‑Plan Answers)

This notebook ingests **multiple 401(k) text files** (three plans) from your local folder:

```
/home/shreyo/MCA-STR-AI/textfiles
├─ Joe_frazier.txt
├─ MCA_adoption_agreement_knight_train_401k.txt
└─ Tristate_manufacturing.txt
```

It builds a **separate FAISS index per plan** with metadata (`plan_name`, `section`, `page`) and uses:
- **Amazon Titan Embeddings** via **AWS Bedrock**
- **Cross‑encoder reranking** using `BAAI/bge-reranker-large`
- **Claude 3 Haiku** (Bedrock) to compose grounded answers

When a **single question** applies to **all plans** (with different values in each), you’ll get **one answer per plan**, clearly labeled.

> ✅ Paths are already set for your machine.  
> ✅ You can optionally persist indices to disk.



## 1) Install dependencies

Run the following once in your environment. (Skip if already installed.)


In [1]:

# If needed, uncomment and run:
# !pip install --upgrade boto3 faiss-cpu numpy transformers torch langchain langchain-community
# Bedrock access requires configured AWS credentials with bedrock:* permissions.



## 2) Imports & configuration

**Credentials**: Ensure `aws configure` is set up or environment variables are present:
- `AWS_ACCESS_KEY_ID`
- `AWS_SECRET_ACCESS_KEY`
- `AWS_DEFAULT_REGION` (or use the `BEDROCK_REGION` variable below)


In [2]:

import os
import json
import glob
import gc
from dataclasses import dataclass
from typing import Dict, List, Tuple

import boto3
import numpy as np
import faiss
import torch

from transformers import AutoModelForSequenceClassification, AutoTokenizer

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage

# -------------------------------
# Config: paths & models
# -------------------------------
TEXTFILES_DIR = "/home/shreyo/MCA-STR-AI/textfiles"   # <-- Your folder
BEDROCK_REGION = "us-east-1"

EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"
LLM_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"

# Chunking (tuned for long sections)
CHUNK_SIZE = 3600
CHUNK_OVERLAP = 300

# Retrieval settings
TOP_K_PER_PLAN = 3   # default top-k chunks per plan for answering

# Torch device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Bedrock runtime client
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=BEDROCK_REGION)

# Embeddings + LLM
bedrock_embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id=EMBEDDING_MODEL_ID
)

llm = BedrockChat(
    client=bedrock_runtime,
    model_id=LLM_MODEL_ID,
    model_kwargs={
        "max_tokens": 2048,
        "temperature": 0.1,
        "top_p": 0.9
    }
)

print(f"Using device: {DEVICE}")


/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


/tmp/ipykernel_7758/805079898.py:43: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  bedrock_embeddings = BedrockEmbeddings(
/tmp/ipykernel_7758/805079898.py:48: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import ChatBedrock``.
  llm = BedrockChat(



## 3) Helpers: parsing, chunking, embeddings, FAISS

We parse the adoption agreements into **(section, page, content)** triplets, split into overlapping chunks, and store metadata:
- `plan_name`
- `section`
- `page`


In [3]:

@dataclass
class Chunk:
    """A single chunk of text with minimal metadata."""
    text: str
    metadata: Dict[str, str]  # keys: plan_name, section, page


def infer_plan_name_from_file_path(file_path: str) -> str:
    """Infer a human-friendly plan name from the filename.
    
    If your files contain an explicit 'Plan Name:' line, you can extend this 
    function to parse the file to find it; otherwise the filename is used.
    """
    base = os.path.basename(file_path)
    name = os.path.splitext(base)[0]
    # Make it look nice: underscores/hyphens -> spaces, title case
    name = name.replace('_', ' ').replace('-', ' ').strip().title()
    return name


def load_and_process_document(file_path: str) -> List[Dict[str, str]]:
    """Load a text file and convert into structured sections.
    
    The function looks for:
      - page markers like: 'Page_5'
      - numbered sections: '1.' or '1 ' up to '57.' / '57 '
    
    Returns a list of dicts: {section, page, content}
    """
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()

    sections = []
    current_section = None
    current_content = []
    current_page = None

    lines = content.splitlines()
    for raw in lines:
        line = raw.strip()
        if not line:
            continue

        if line.startswith('Page_'):
            # Track page number marker
            current_page = line.replace('Page_', '').strip()

        # Sections labeled "1. ..." or "1 ..." up to "57"
        elif any(line.startswith(f'{i}.') for i in range(1, 58)) or              any(line.startswith(f'{i} ') for i in range(1, 58)):
            if current_section and current_content:
                sections.append({
                    'section': current_section,
                    'page': current_page,
                    'content': ' '.join(current_content)
                })
            current_section = line
            current_content = []
        else:
            if current_section:
                current_content.append(line)

    # flush the last section
    if current_section and current_content:
        sections.append({
            'section': current_section,
            'page': current_page,
            'content': ' '.join(current_content)
        })

    return sections


def chunk_sections(sections: List[Dict[str, str]], plan_name: str) -> List[Chunk]:
    """Split each section into overlapping chunks and attach metadata."""
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

    chunks: List[Chunk] = []
    for sec in sections:
        for text in splitter.split_text(sec['content']):
            chunks.append(
                Chunk(
                    text=text,
                    metadata={
                        'plan_name': plan_name,
                        'section': sec['section'],
                        'page': sec.get('page', None) or 'NA'
                    }
                )
            )
    return chunks


def build_faiss_index(chunks: List[Chunk]) -> Tuple[faiss.IndexFlatL2, np.ndarray]:
    """Create FAISS L2 index for a list of chunks using Titan embeddings."""
    texts = [c.text for c in chunks]
    if not texts:
        raise ValueError("No texts to embed for FAISS index.")

    embeddings = bedrock_embeddings.embed_documents(texts)
    dim = len(embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings, dtype='float32'))
    return index, np.array(embeddings, dtype='float32')



## 4) Ingest all 3 plans and build **per‑plan FAISS indices**

We’ll create a dictionary:
```python
indices_by_plan = {
    plan_name: {
        "index": <faiss.Index>,
        "embeddings": np.ndarray,
        "chunks": List[Chunk]
    }
}
```


In [4]:

# Discover .txt files in the folder
txt_files = sorted(glob.glob(os.path.join(TEXTFILES_DIR, "*.txt")))
if not txt_files:
    raise FileNotFoundError(f"No .txt files found in {TEXTFILES_DIR}")

indices_by_plan: Dict[str, Dict[str, object]] = {}

for file_path in txt_files:
    plan_name = infer_plan_name_from_file_path(file_path)
    print(f"Processing plan: {plan_name}  |  File: {file_path}")

    sections = load_and_process_document(file_path)
    print(f"  - Sections detected: {len(sections)}")

    chunks = chunk_sections(sections, plan_name=plan_name)
    print(f"  - Chunks created: {len(chunks)}")

    index, emb = build_faiss_index(chunks)
    print(f"  - FAISS index size: {index.ntotal}")

    indices_by_plan[plan_name] = {
        "index": index,
        "embeddings": emb,
        "chunks": chunks
    }

print(f"\nBuilt indices for {len(indices_by_plan)} plans: {list(indices_by_plan.keys())}")


Processing plan: Joe Frazier  |  File: /home/shreyo/MCA-STR-AI/textfiles/Joe_frazier.txt
  - Sections detected: 55
  - Chunks created: 73
  - FAISS index size: 73
Processing plan: Mca Adoption Agreement Knight Train 401K  |  File: /home/shreyo/MCA-STR-AI/textfiles/MCA_adoption_agreement_knight_train_401k.txt
  - Sections detected: 59
  - Chunks created: 77
  - FAISS index size: 77
Processing plan: Tristate Manufacturing  |  File: /home/shreyo/MCA-STR-AI/textfiles/Tristate_manufacturing.txt
  - Sections detected: 57
  - Chunks created: 75
  - FAISS index size: 75

Built indices for 3 plans: ['Joe Frazier', 'Mca Adoption Agreement Knight Train 401K', 'Tristate Manufacturing']



## 5) Cross‑encoder reranker (optional but recommended)

We use **`BAAI/bge-reranker-large`** to re-score the retrieved chunks for each plan.  
This improves precision by evaluating *(query, passage)* pairs.


In [10]:

# Load cross-encoder reranker
RERANKER_MODEL_NAME = "BAAI/bge-reranker-large"

reranker_tokenizer = AutoTokenizer.from_pretrained(RERANKER_MODEL_NAME)
reranker_model = AutoModelForSequenceClassification.from_pretrained(RERANKER_MODEL_NAME)
reranker_model.to(DEVICE)
reranker_model.eval()

@torch.no_grad()
def rerank(query: str, candidate_chunks: List[Chunk], top_k: int) -> List[Chunk]:
    """Rerank candidate chunks using cross-encoder scores and return top_k."""
    if not candidate_chunks:
        return []

    pairs = [(query, c.text) for c in candidate_chunks]
    inputs = reranker_tokenizer(
        pairs,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    scores = reranker_model(**inputs).logits.view(-1).float()
    sorted_idx = torch.argsort(scores, descending=True).tolist()
    return [candidate_chunks[i] for i in sorted_idx[:top_k]]



## 6) Retrieval & prompting per plan

We retrieve **top‑k per plan** (via FAISS), **rerank**, and then prompt the LLM with a **plan‑specific context**.  
The LLM is instructed to include **section & page references**.


In [ ]:

def retrieve_per_plan(query: str, plan_name: str, k: int = TOP_K_PER_PLAN) -> List[Chunk]:
    """Retrieve top-k chunks for a specific plan via FAISS, then rerank with cross-encoder."""
    plan_data = indices_by_plan[plan_name]
    index: faiss.IndexFlatL2 = plan_data["index"]
    chunks: List[Chunk] = plan_data["chunks"]

    q_emb = bedrock_embeddings.embed_query(query)
    D, I = index.search(np.array([q_emb], dtype='float32'), k * 5)  # overfetch then rerank
    initial = [chunks[i] for i in I[0] if i >= 0]
    reranked = rerank(query, initial, top_k=k)
    return reranked


def build_context_for_prompt(chunks: List[Chunk]) -> str:
    """Create a readable context string with plan references."""
    parts = []
    for c in chunks:
        parts.append(
            f"From Plan {c.metadata['plan_name']} — Section {c.metadata['section']} (Page {c.metadata['page']}):\n{c.text}"
        )
    return "\n\n".join(parts)


def create_prompt(query: str, context_chunks: List[Chunk]) -> str:
    """Compose a checkbox-aware, per-plan JSON prompt for the LLM with strict election logic."""
    context = build_context_for_prompt(context_chunks)
    prompt = f"""
You are a compliance-grade 401(k) Adoption Agreement analyst. Your responses must reflect only what is elected via checkboxes and filled text in the provided excerpts.

Each excerpt is labeled: "Plan: <PLAN_NAME> — Section <LABEL> (Page <PAGE>): <TEXT>"

### 🔴 ABSOLUTE RULES (NON-NEGOTIABLE)

1) ✅ **CHECKBOX LOGIC IS LAW**:
   - "[x]" or "[X]" = SELECTED
   - "[ ]" = NOT SELECTED
   - If a section says "Choose one of (a) or (b)", and (a) is checked, then (b) **does not apply**, even if text from (b) is present.
   - If a section says "Do not apply" and it is checked, then the entire feature is **OFF** — do **not** describe rates, formulas, or schedules.

2) 🚫 **NO HALLUCINATIONS**:
   - If a feature is disabled (e.g., "Do not apply" is checked), do **not** describe its parameters.
   - If no percentage, formula, or amount is specified, return "not specified" — do **not** invent numbers.

3) 📌 **FOLLOW SKIP INSTRUCTIONS**:
   - If a line says "(skip to Election X)" and the option is checked, then all later elections (e.g., 21(b), 21(3), 34) are **irrelevant and not in effect**.

4) 🔍 **CONTEXT IS KING**:
   - Do **not** use outside knowledge (IRS rules, common practices, defaults).
   - If something is not explicitly checked or filled, it is **not part of the plan**.

5) 🧩 PER-PLAN ISOLATION:
   - Answer for each plan **independently**. Never assume one plan’s election applies to another.

6) 📎 CITATIONS:
   - Cite only the section/election label and page where the answer is found.
   - If multiple citations, list all that directly support the answer.

7) 📏 NORMALIZATION:
   - Convert terms: "3 months" → "3 months", "1,000 hours" → "1,000 Hours of Service", "50%" → "50% of Compensation".

---

### TASK
Answer the question using only the provided context. Return **only** a valid JSON object.

### OUTPUT SCHEMA
{{
  "question": "{query}",
  "plan_answers": [
    {{
      "plan_name": "<string>",
      "value": "<concise factual answer>",
      "explanation": "<1-3 sentences. Reference only checked boxes and filled text.>",
      "status": "found | not_specified",
      "citations": [
        {{"section_or_election": "<label>", "page": "<page>"}}
      ]
    }}
  ],
  "comparison_summary": "<1-2 sentence factual comparison across plans. Do not add recommendations.>"
}}

---

### CONTEXT
{context}

### QUESTION
{query}

Return ONLY the JSON object. No extra text.
"""
    return prompt

def answer_for_plan(query: str, plan_name: str, k: int = TOP_K_PER_PLAN) -> Tuple[str, List[Chunk]]:
    """Retrieve, rerank, and answer for a single plan. Returns (answer_text, used_chunks)."""
    context_chunks = retrieve_per_plan(query, plan_name, k=k)
    if not context_chunks:
        return "I don't know.", []

    prompt = create_prompt(query, context_chunks)
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content, context_chunks



## 7) Orchestrator: answer across **all plans**

This returns a **list of per‑plan answers**, each including the text and the chunks used.


In [12]:

def answer_question_multi(query: str, k: int = TOP_K_PER_PLAN) -> List[Dict[str, object]]:
    """Answer a query for ALL plans, returning a list of dicts:
    [ { 'plan_name': str, 'answer': str, 'references': [(section, page), ...] }, ... ]
    """
    results = []
    for plan_name in indices_by_plan.keys():
        ans, chunks = answer_for_plan(query, plan_name, k=k)
        refs = [(c.metadata['section'], c.metadata['page']) for c in chunks]
        results.append({
            "plan_name": plan_name,
            "answer": ans,
            "references": refs
        })
    return results


def pretty_print_results(results: List[Dict[str, object]]):
    """Nicely print per‑plan answers in the notebook output."""
    for r in results:
        print("=" * 90)
        print(f"Plan: {r['plan_name']}")
        print("-" * 90)
        print(r['answer'].strip())
        print("\nCited sections/pages:")
        for (sec, pg) in r['references']:
            print(f"  - {sec} (Page {pg})")
        print("=" * 90 + "\n")



## 8) (Optional) Persist & reload indices

You can save each plan’s FAISS index and its chunk metadata for reuse.


In [13]:

PERSIST_DIR = "./faiss_indices"

def persist_indices(indices: Dict[str, Dict[str, object]], persist_dir: str = PERSIST_DIR):
    os.makedirs(persist_dir, exist_ok=True)
    for plan_name, data in indices.items():
        # Save FAISS index
        idx_path = os.path.join(persist_dir, f"{plan_name}.faiss")
        faiss.write_index(data["index"], idx_path)

        # Save chunks metadata
        meta_path = os.path.join(persist_dir, f"{plan_name}_chunks.json")
        serializable = [dict(text=c.text, metadata=c.metadata) for c in data["chunks"]]
        with open(meta_path, "w", encoding="utf-8") as f:
            json.dump(serializable, f, ensure_ascii=False)

    print(f"Saved indices & metadata to: {os.path.abspath(persist_dir)}")


def load_indices(persist_dir: str = PERSIST_DIR) -> Dict[str, Dict[str, object]]:
    loaded = {}
    for idx_file in glob.glob(os.path.join(persist_dir, "*.faiss")):
        plan_name = os.path.splitext(os.path.basename(idx_file))[0]
        index = faiss.read_index(idx_file)

        meta_file = os.path.join(persist_dir, f"{plan_name}_chunks.json")
        if not os.path.exists(meta_file):
            print(f"Warning: missing metadata for {plan_name}, skipping.")
            continue

        with open(meta_file, "r", encoding="utf-8") as f:
            meta = json.load(f)
        chunks = [Chunk(text=m['text'], metadata=m['metadata']) for m in meta]

        loaded[plan_name] = {
            "index": index,
            "embeddings": None,   # not required for search
            "chunks": chunks
        }
    return loaded



## 9) Try it out: sample questions

These are example questions (you can modify or add more). The output will show **one answer per plan**.


In [14]:

sample_questions = [
    "At what age is an employee eligible to participate in the plan?",
    "How long must an employee work to be eligible to participate in the plan?",
    "When can an employee begin contributing to the plan?",
    "Are there limits on the contributions/deferrals employee participants can make to the plan?",
    "What type of contributions may an employee make to the plan?",
    "What type of contributions will my employer make to the plan?",
    "How much are participants matched on their contributions?",
    "Does the plan have automatic deferrals?",
    "What is the automatic deferral rate?",
]

# Example: run all and print per‑plan answers
for i, q in enumerate(sample_questions, 1):
    print(f"\n### Question {i}: {q}\n")
    results = answer_question_multi(q, k=TOP_K_PER_PLAN)
    pretty_print_results(results)



### Question 1: At what age is an employee eligible to participate in the plan?



/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Plan: Joe Frazier
------------------------------------------------------------------------------------------
{
  "question": "At what age is an employee eligible to participate in the plan?",
  "plan_answers": [
    {
      "plan_name": "Joe Frazier",
      "value": "18 years old",
      "explanation": "The plan specifies that the eligibility condition for all contribution types is age 18 or older.",
      "status": "found",
      "citations": [
        {
          "section_or_election": "14(e)",
          "page": "NA"
        }
      ]
    }
  ],
  "comparison_summary": "The plan specifies that the eligibility age for all contribution types is 18 years old."
}

Cited sections/pages:
  - 14. ELIGIBILITY (2.01). To become a Participant in the Plan, an Eligible Employee must satisfy (Choose one of (a), (b), or (c).): (Page NA)
  - 39. NORMAL RETIREMENT AGE (5.01). A Participant attains Normal Retirement Age under the Plan on the following date (Choose one of (a) or (b).): (Page NA)
  - 1

/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Plan: Joe Frazier
------------------------------------------------------------------------------------------
{
  "question": "How long must an employee work to be eligible to participate in the plan?",
  "plan_answers": [
    {
      "plan_name": "Joe Frazier",
      "value": "1 year of service",
      "explanation": "The plan requires 1 year of service (1,000 hours) for an employee to be eligible to participate in the plan.",
      "status": "found",
      "citations": [
        {
          "section_or_election": "Section 14. ELIGIBILITY (2.01)",
          "page": "NA"
        },
        {
          "section_or_election": "Section 16. YEAR OF SERVICE - ELIGIBILITY (2.02(A))",
          "page": "NA"
        }
      ]
    }
  ],
  "comparison_summary": "The plan requires 1 year of service (1,000 hours) for an employee to be eligible to participate in the plan."
}

Cited sections/pages:
  - 18. PROSPECTIVE/RETROACTIVE ENTRY DATE (2.02(D)). An Employee after satisfying the eligibility con

...

Plan: Joe Frazier
------------------------------------------------------------------------------------------
{
  "question": "How much are participants matched on their contributions?",
  "plan_answers": [
    {
      "plan_name": "Joe Frazier",
      "value": "100% of the first 3% of participant contributions, plus 50% of the next 2% of participant contributions",
      "explanation": "The plan provides a Basic Matching Contribution equal to 100% of each participant's elective deferrals not exceeding 3% of the participant's compensation, plus 50% of each participant's elective deferrals in excess of 3% but not in excess of 5% of the participant's compensation.",
      "status": "found",
      "citations": [
        {
          "section_or_election": "30(c)",
          "page": "NA"
        }
      ]
    }
  ],
  "comparison_summary": "The plan provides a Basic Matching Contribution formula, which is a common matching structure in 401(k) plans."
}

Cited sections/pages:
  - 36. EMPLOYEE

/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Plan: Joe Frazier
------------------------------------------------------------------------------------------
{
  "question": "Does the plan have automatic deferrals?",
  "plan_answers": [
    {
      "plan_name": "Joe Frazier",
      "value": "Yes",
      "explanation": "The plan has automatic deferrals, specifically an Automatic Contribution Arrangement (ACA) under Section 3.02(B)(1). The automatic deferral percentage is set at 2% of the participant's compensation, with no scheduled increases.",
      "status": "found",
      "citations": [
        {
          "section_or_election": "Section 21. AUTOMATIC DEFERRAL (ACA/EACA/QACA) (3.02(B))",
          "page": "NA"
        }
      ]
    }
  ],
  "comparison_summary": "The plan has automatic deferrals in the form of an ACA with a fixed 2% deferral rate."
}

Cited sections/pages:
  - 21. AUTOMATIC DEFERRAL (ACA/EACA/QACA) (3.02(B)). The Automatic Deferral provisions of Section 3.02(B) (Choose one of (a) or (b). Also see Election 34 regar

/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/home/shreyo/miniconda3/envs/insurance-rag-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Plan: Joe Frazier
------------------------------------------------------------------------------------------
{
  "question": "What is the automatic deferral rate?",
  "plan_answers": [
    {
      "plan_name": "Joe Frazier",
      "value": "2%",
      "explanation": "The plan has a fixed automatic deferral percentage of 2% of the participant's compensation, as specified in Section 21(b)(3)a.",
      "status": "found",
      "citations": [
        {
          "section_or_election": "Section 21. AUTOMATIC DEFERRAL (ACA/EACA/QACA) (3.02(B))",
          "page": "NA"
        }
      ]
    }
  ],
  "comparison_summary": "The plan has a fixed automatic deferral rate of 2% of the participant's compensation."
}

Cited sections/pages:
  - 21. AUTOMATIC DEFERRAL (ACA/EACA/QACA) (3.02(B)). The Automatic Deferral provisions of Section 3.02(B) (Choose one of (a) or (b). Also see Election 34 regarding Automatic Escalation of Salary Reduction Agreements.): (Page NA)
  - 21. AUTOMATIC DEFERRAL (ACA/EAC


## 10) Troubleshooting tips

- **No Bedrock access**: Make sure your AWS account has Bedrock enabled in `us-east-1` and you’ve accepted model access for Titan & Claude.
- **Credentials**: Run `aws configure` or set environment variables for credentials/region.
- **Large reranker model**: `BAAI/bge-reranker-large` is sizable. If memory is tight, switch to `BAAI/bge-reranker-base`.
- **Parsing**: If your files format sections differently, update `load_and_process_document` accordingly.
- **Speed**: For quick tests, reduce `CHUNK_SIZE` and `TOP_K_PER_PLAN`.
